# AWS CodePipeline で ML パイプラインを更新

## 背景

01-sagemaker-training-inference-pipeline で作成した Step Functions Workflow を AWS CodePipeline, CodeBuild, CodeCommit を使って更新するパイプラインを作成します。CodeCommit のファイルが更新されると、それをトリガーにして Step Functions Workflow の作成やコンテナイメージの作成が実行されます。

## セットアップ
前に使用したノートブックからパラメタを引き継ぎます。

In [ ]:
%store -r

In [ ]:
import boto3
import json
import os
import pandas as pd
import sagemaker
from sagemaker.processing import Processor, ProcessingInput, ProcessingOutput
from time import sleep
import utility

%store -r

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
codecommit_client = boto3.client('codecommit', region_name=region)
codebuild_client = boto3.client('codebuild', region_name=region)
codepipeline_client = boto3.client('codepipeline', region_name=region)

role = sagemaker.get_execution_role()
account_id = boto3.client('sts').get_caller_identity().get('Account')

policy_arn_list = []
role_name_list = []

code_dir = 'repo'

codepipeline_name  = project_name + '-ml-codepipeline-' + user_name
codebuild_project_name  = project_name + '-ml-codebuild-' + user_name
code_repository_name  = project_name + '-ml-repo-' + user_name

## pipeline-config.yml の作成

Step Functions Workflow 作成の際に必要なパラメタを yaml ファイルに保存します。このファイルは CodeCommit で管理します。

In [ ]:
filepath = os.path.join(code_dir, 'pipeline-config.yml')
prep_repository_name =  prep_repository_uri.split('/')[1].split(':')[0]
train_repository_name =  train_repository_uri.split('/')[1].split(':')[0]

config_yml={f"""
config:
  user-name: {user_name}
  region: {region}
  bucket-name: {bucket_name}
  s3-prefix: {prefix}
  num-of-segment: {num_of_segment}
  codepipeline-name: {codepipeline_name}
  prep-image-name: {prep_repository_name}
  train-image-name: {train_repository_name}
  sfn-workflow-arn: {step_functions_workflow_arn}
  sfn-role-arn: {workflow_execution_role}
  sagemaker-role: {role}
  notification-lambda-name: {lambda_notification_function_name}
  sns-topic-arn: {sns_notification_topic_arn}
"""
}

with open(filepath, 'w') as f:
    f.write('\n'.join(list(config_yml)))

## AWS CodeCommit リポジトリの作成

In [ ]:
try:
    response = codecommit_client.create_repository(
        repositoryName=code_repository_name,
        repositoryDescription='sample repository',
        tags={
            'owner': user_name
        }
    )
except Exception as e:
    print(e)

作成したリポジトリにSageMaker Job 関連のファイルを push します。

In [ ]:
%%sh -s $code_repository_name $region $user_name
echo $1
rm -rf $1
git clone https://git-codecommit.$2.amazonaws.com/v1/repos/$1
cd $1
git config --global user.name $3
git config --global user.email you@example.com
git checkout -b main
cp -r ../code ./
cp -r ../docker ./
cp -r ../repo/* ./
git add .
git commit -m "first commit"
git push --set-upstream origin main

以下のセルを実行して表示されたリンクをクリックし、ファイルが push されたか確認してみましょう。

In [ ]:
from IPython.display import display, Markdown
display(Markdown(f"<a href=\"https://{region}.console.aws.amazon.com/codesuite/codecommit/repositories/{code_repository_name}/setup?region={region}\" target=\"_blank\">CodeCommit リポジトリ</a>"))


## AWS CodeBuild プロジェクトの作成

以下のセルを実行して、CodeBuild が使用する IAM Policy と Role を作成します。

In [ ]:
assume_role_policy = {
          "Version": "2012-10-17",
          "Statement": [{"Sid": "","Effect": "Allow","Principal": {"Service":"codebuild.amazonaws.com"},"Action": "sts:AssumeRole"}]
        }

codebuild_policy_name = codebuild_project_name + '-policy'
codebuild_role_name = codebuild_project_name + '-role'
codebuild_policy_json_name = 'codebuild-policy.json'

codebuild_role_arn = utility.create_policy_role(codebuild_policy_name, codebuild_policy_json_name,
                    codebuild_role_name, assume_role_policy,
                    role_name_list, policy_arn_list)

Step Functions Workflow を更新するための CodeBuild プロジェクトを作成します。

In [ ]:
response = codebuild_client.create_project(
    name=codebuild_project_name,
    description='sample project',
    source={
        'type': 'CODEPIPELINE',
        'insecureSsl': False,
    },
    artifacts={
        'type': 'CODEPIPELINE',
        'encryptionDisabled': False,
    },
    cache={
        'type': 'LOCAL',
        'modes': [
            'LOCAL_DOCKER_LAYER_CACHE',
        ]
    },
    environment={
        'type': 'LINUX_CONTAINER',
        'image': 'aws/codebuild/standard:5.0',
        'computeType': 'BUILD_GENERAL1_MEDIUM',
        'privilegedMode': True,
        'imagePullCredentialsType': 'CODEBUILD'
    },
    serviceRole=codebuild_role_arn,
    timeoutInMinutes=120,
    queuedTimeoutInMinutes=120,
#     encryptionKey='string',
    tags=[
        {
            'key': 'owner',
            'value': user_name
        },
    ],
    badgeEnabled=False,
    logsConfig={
        'cloudWatchLogs': {
            'status': 'ENABLED',
        },
        's3Logs': {
            'status': 'ENABLED',
            'location': os.path.join(bucket_name, 'build-log'),
            'encryptionDisabled': False,
            'bucketOwnerAccess': 'READ_ONLY'
        }
    },
)

## CodePipeline のパイプライン作成

まずは CodePipeline のパイプラインにアタッチする IAM Role を作成します。

In [ ]:
codepipeline_policy_name = codepipeline_name + '-policy'
codepipeline_role_name = codepipeline_name + '-role'
codepipeline_role_json_name = 'codepipeline-policy.json'

assume_role_policy = {
      "Version": "2012-10-17",
      "Statement": [{"Sid": "","Effect": "Allow","Principal": {"Service":"codepipeline.amazonaws.com"},"Action": "sts:AssumeRole"}]
    }

codepipeline_role_arn = utility.create_policy_role(
                            codepipeline_policy_name, codepipeline_role_json_name,
                            codepipeline_role_name, assume_role_policy,
                            role_name_list, policy_arn_list)


In [ ]:
def get_pipeline_list():
    file_list = []
    next_token = ''
    while True:
        if next_token == '':
            response = codepipeline_client.list_pipelines()
        else:
            response = codepipeline_client.list_pipelines(nextToken=next_token)
        for content in response['pipelines']:
            key = content['name']
            file_list.append(key)
        if 'nextToken' in response:
            next_token = response['nextToken']
        else:
            break

    return file_list

CodePipeline のパイプラインを作成します。Source Stage と Build Stage を作成します。

In [ ]:
pipeline={
    'name': codepipeline_name,
    'roleArn': codepipeline_role_arn,
    'artifactStore': {
        'type': 'S3',
        'location': bucket_name,
    },
    'stages': [
        {
            'name': 'Source',
            'actions': [
                {
                    'name': 'Code',
                    'actionTypeId': {
                        'category': 'Source',
                        'owner': 'AWS',
                        'provider': 'CodeCommit',
                        'version': '1'
                    },
                    'runOrder': 1,
                    'configuration': {
                        'BranchName': "main", 
                        'PollForSourceChanges': "true",
                        "OutputArtifactFormat": "CODEBUILD_CLONE_REF",
                        'RepositoryName': code_repository_name
                    },
                    'outputArtifacts': [
                        {
                            'name': 'Code'
                        },
                    ],
                },
            ]
        },
        {
            'name': 'Build',
            'actions': [
                {
                    'name': 'CreateMLWorkflow',
                    'actionTypeId': {
                        'category': 'Build',
                        'owner': 'AWS',
                        'provider': 'CodeBuild',
                        'version': '1'
                    },
                    'runOrder': 2,
                    # CodeBuild の configuration 情報はこちら
                    # https://docs.aws.amazon.com/codepipeline/latest/userguide/action-reference-CodeBuild.html
                    'configuration': {
                        "BatchEnabled": "false",
                        "CombineArtifacts": "false",
                        "ProjectName": codebuild_project_name,
                        "PrimarySource": "Code",
                        "EnvironmentVariables": 
                            "[{\"name\":\"EXEC_ID\","
                                "\"value\":\"#{codepipeline.PipelineExecutionId}\","
                                "\"type\":\"PLAINTEXT\"}]"
                    },
                    'inputArtifacts': [
                        {
                            'name': 'Code'
                        },
                    ],
                    'namespace': 'CreateMLWorkflow'
                }
            ]
        }],
    'version': 1
}

if codepipeline_name in get_pipeline_list():
    response = codepipeline_client.update_pipeline(
        pipeline=pipeline
    )
else:
    response = codepipeline_client.create_pipeline(
        pipeline=pipeline,
        tags=[
            {
                'key': 'owner',
                'value': user_name
            },
        ]
    )

CodePipeline のパイプラインが作成されました。パイプラインが自動的に開始しているはずなので、以下のセルを実行して表示されたリンクをクリックして CodePipeline のコンソールで様子を確認しましょう。Source Stage -> Build Stage の順でパイプラインが実行されます。

Build Stage の実行には最長で 10分ほどかかります。コンテナイメージ関連のファイルの更新がない場合はコンテナイメージのビルドがスキップされるため 3分ほどで完了します。

In [ ]:
from IPython.display import display, Markdown
display(Markdown(f"<a href=\"https://{region}.console.aws.amazon.com/codesuite/codepipeline/pipelines/{codepipeline_name}/view?region={region}\" target=\"_blank\">CodePipeline のコンソール</a>"))

## [Optional] CodeCommit のファイルの更新

このノートブックと同じフォルダにある code_repository_name と同じ名前のフォルダの中のソースコードを変更したら、以下のセルを実行して変更を CodeCommit リポジトリに反映します。ファイルの変更がリポジトリに反映されると CodePipeline が開始します。

In [ ]:
%%sh -s $code_repository_name
cd $1
git add .
git commit -m "update"
git push

これで、必要なリソースをすべて作成することができました。これらのリソースが不要になった場合は、忘れずに以下のリソースの削除を実施してから、[こちらのリソース削除](01-sagemaker-training-inference-pipeline.ipynb#リソースの削除) を実行してください。

## リソースの削除

作成したリソースが不要になったら以降のセルを実行して、このノートブックで作成したリソースを削除してください。

### CodeCommit, CodeBuild, CodePipeline の削除

In [ ]:
response = codecommit_client.delete_repository(
    repositoryName=code_repository_name
)
print('Delete:', response['repositoryId'])

response = codebuild_client.delete_project(
    name=codebuild_project_name
)
print('Delete:', codebuild_project_name)

response = codepipeline_client.delete_pipeline(
    name=codepipeline_name
)
print('Delete:', codepipeline_name)


### IAM Role と Policy の削除

In [ ]:
role_name_list = list(set(role_name_list))
policy_arn_list = list(set(policy_arn_list))

utility.delete_role_policy(role_name_list, policy_arn_list)